If you are using VSCode, simply clone the repository in a container volume. Otherwise, you'll need standard Anaconda packages as well as [CVXPY](https://www.cvxpy.org/install/).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Helper function for later

In [2]:
def get_dispatch(market, intercept, slope):
    return np.maximum(
        np.minimum(
            intercept + slope*market.Bid - market.Capacity.cumsum().shift(fill_value=0),
            market.Capacity
        ),
        0
    )

Load and process portfolio data

In [3]:
portfolios = pd.read_excel('ESG Portfolios.xlsx')
portfolios.columns = portfolios.iloc[0:3].bfill().iloc[0]
portfolios = portfolios.iloc[3:]
portfolios = portfolios[portfolios['UNIT NAME'] != 'Totals']
portfolios.dropna(axis='index', how='all', inplace=True)
portfolios.dropna(axis='columns', how='all', inplace=True)
portfolios['portfolio'] = (portfolios.isna().any(axis='columns') * portfolios['UNIT NAME']).replace('', np.NaN).ffill()
portfolios.dropna(axis='index', how='any', inplace=True)
portfolios.set_index(['Location', 'portfolio', 'UNIT NAME'], inplace=True)
portfolios.sort_index(axis='index',level=['Location', 'portfolio'], inplace=True)

Load and process profile data

In [4]:
profiles = pd.read_excel('ESG Demand Year 2.xlsx', header=2, index_col=[1,0,2])
profiles.dropna(axis='index', how='any', inplace=True)
profiles.sort_index(inplace=True)

For now, we'll just simulate the first hour of the first day. Edit this line to choose a different hour and/or day.

In [5]:
hour = profiles.loc[(['North', 'South'], 1, 1)].droplevel(['Day', 'Hour'])

This cell adds a column to the ``portfolio`` dataframe containing the dispatched generation, computed according to the bids. See the comments in the cell for ways to modify the simulation.

In [6]:
# Set the bids. 
# The name of the column must remain 'Bid'.
# To test a different strategy than marginal cost, edit the right-hand side.
portfolios['Bid'] = portfolios['Total Marginal Cost']

# Set the transmission capacity
transmission_capacity = 10000

portfolios.sort_values('Bid', inplace=True)

intercept, slope = hour.sum()

portfolios['Dispatch'] = get_dispatch(portfolios, intercept, slope)

# Compute generation and dispatch at each location
generation = portfolios.Dispatch.groupby('Location').sum()
consumption = np.linalg.inv(
    np.array(
        [
            np.array([1,-1])*np.array([1/hour.loc[location, 'Load Slope'] for location in hour.index]),
            [1, 1]
        ]
    )
)@np.array(
    [
        np.array([1, -1])@(hour['Load Intercept']/hour['Load Slope']),
        portfolios.Dispatch.sum()
    ]
)

shortage = consumption - generation

if np.abs(shortage).unique().squeeze() >= transmission_capacity:

    shortage = transmission_capacity*np.sign(shortage)
    portfolios.Dispatch = portfolios.groupby('Location', group_keys=False).apply(
        lambda market: get_dispatch(market, hour.loc[market.name, 'Load Intercept'] - shortage[market.name], hour.loc[market.name, 'Load Slope'])
    )
    generation = portfolios.Dispatch.groupby('Location').sum()
    consumption = generation + shortage

price = (consumption - hour['Load Intercept'])/hour['Load Slope']

Check the results. To check profit, compute the price by evaluating the marginal utility function for each region at the consumption in that region. The consumption by region is contained in the variable ``consumption``.

In [8]:
portfolios.groupby(level='Location').apply(lambda x: x)

0                                                       Fuel Capacity  \
Location Location portfolio    UNIT NAME                                
North    North    Fossil Light HELMS                   Hydro      800   
                               DIABLO CANYON 1       Nuclear     1000   
                  Bay Views    MOSS LANDING 6            Gas      750   
                               MOSS LANDING 7            Gas      750   
                               MORRO BAY 3&4             Gas      665   
                  East Bay     PITTSBURGH 5&6            Gas      650   
                  Bay Views    MORRO BAY 1&2             Gas      335   
                  East Bay     CONTRA COSTA 6&7          Gas      700   
                               PITTSBURGH 1-4            Gas      650   
                  Fossil Light HUMBOLDT                  Gas      150   
                               HUNTERS POINT 1&2         Gas      150   
                  East Bay     CONTRA COSTA 4&5          Gas      150   
                               PITTSBURGH 7              Gas      700   
                  Bay Views    OAKLAND                   Gas      150   
                  East Bay     POTRERO HILL              Gas      150   
                  Fossil Light HUNTERS POINT 4           Gas      250   
South    South    Old Timers   BIG CREEK               Hydro     1000   
                               MOHAVE 1                 Coal      750   
                               MOHAVE 2                 Coal      750   
                  Big Coal     FOUR CORNERS             Coal     1900   
                  Beachfront   ORMOND BEACH 2            Gas      700   
                               ORMOND BEACH 1            Gas      700   
                               MANDALAY 1&2              Gas      300   
                  Big Coal     HUNTINGTON BEACH 1&2      Gas      300   
                  Big Gas      EL SEGUNDO 3&4            Gas      650   
                               ENCINA                    Gas      950   
                  Big Coal     REDONDO 5&6               Gas      350   
                               REDONDO 7&8               Gas      950   
                  Beachfront   COOLWATER                 Gas      650   
                               ETIWANDA 1-4              Gas      850   
                  Big Gas      SOUTH BAY                 Gas      700   
                               EL SEGUNDO 1&2            Gas      400   
                  Old Timers   HIGHGROVE                 Gas      150   
                  Beachfront   MANDALAY 3                Gas      150   
                  Big Gas      LONG BEACH                Gas      550   
                  Old Timers   SAN BERNADINO             Gas      100   
                  Beachfront   ETIWANDA 5                Gas      150   
                  Big Gas      NORTH ISLAND              Gas      150   
                  Big Coal     HUNTINGTON BEACH 5        Gas      150   
                               ALAMITOS 7                Gas      250   
                  Beachfront   ELLWOOD                   Gas      300   
                  Big Gas      KEARNY                    Gas      200   

0                                                    Heat Rate Fuel Price  \
Location Location portfolio    UNIT NAME                                    
North    North    Fossil Light HELMS                         0          0   
                               DIABLO CANYON 1               1        7.5   
                  Bay Views    MOSS LANDING 6         6.901235        4.5   
                               MOSS LANDING 7         6.901235        4.5   
                               MORRO BAY 3&4          8.024691        4.5   
                  East Bay     PITTSBURGH 5&6         8.024691        4.5   
                  Bay Views    MORRO BAY 1&2          8.506173        4.5   
                  East Bay     CONTRA COSTA 6&7       8.666667        4.5   
                 